In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import os
import itertools
import statistics

In [2]:
from ipynb.fs.full.particle_file_search import search_particle_filter
from ipynb.fs.full.behavior_segmentation_search import search_behavior_segmentation
from ipynb.fs.full.plot_animals import plot_function

In [1]:
import sys
sys.path.append("/Users/pierre.le.merre/dmclab-kostas/Developer/projects/dmcdb_py")

In [2]:
from readSessionsServer import SessionIterator

In [3]:
state_order = {
    'initRight': 0,
    'initLeft': 0,
    'mainRunLeft': 1,
    'turnMainToLeft': 2,
    'leftRun': 3,
    'leftReward': 4
}

In [4]:

# Code to read the h5 file that contains behavior segmentation
# pathname_2 = input("Insert path of behavior segmentation h5 file here: ")

# with pd.HDFStore(pathname_2) as hdf:
#     # This prints a list of all group names:
#     print("Reading the h5 file that contains the deeplabcut and calcium imaging data...")
#     print(hdf.keys())

# segmented = pd.read_hdf(pathname_2, key="/per_frame")

In [5]:
# Iterate through the particle filter directory and print the pathnames of files and place them in a list
# particle_filter_list = search_particle_filter(dir_path = input("Insert directory path here: "))
# print("Files in folder: \n", particle_filter_list)

In [6]:
# Iterate behavioral segmentation directory
# beh_segmentation_list = search_behavior_segmentation(dir_path = input("Insert directory path here: "))
# print("Files in folder: \n", beh_segmentation_list)

In [ ]:
n_bins_per_seg = 10

#This loops through all the sessions (a.k.a. realizations).
#To just get a single session, write
# `sess = SessionIterator("Miniscope recordings").findSession()`
#instead of the loop.
for sess in SessionIterator("ipsilateral").findSessions():
    #Get the filtered speed and angular speed from the particle filter
    filtered = sess.getProcessedTable("Particle Filter")

    #Get the behavior segmentation per frame
    segmented = sess.getProcessedTable("Behavior segmentation", "per_frame")

    #For each frame, add an int 0-4 to match which phase of the trial the
    # frames falls in, or NaN if it's some other phase not in the list.
    state_order_series = pd.Series(state_order, name="state_order")
    segmented_relevant = segmented.join(state_order_series, on="state_name")

    #The x-coordinate is the state order (0 to 4), plus the progression through 
    #each frame (0 to 1) according to the behavior segmentation
    x_pos = segmented_relevant.spatial_progress + segmented_relevant.state_order

    #Only look at the speeds on frames which are not NaN, i.e. which fall into
    #one of the phases in `state_order`
    speeds = filtered.speed[np.isfinite(x_pos)]

    #Filter out NaNs and bin the x-coordinates
    bin_x = np.floor(x_pos[np.isfinite(x_pos)]*n_bins_per_seg).astype(int)

    #Per bin of x-coordiate...
    gby = (60*speeds).groupby([bin_x, segmented_relevant.segment_no]).mean().groupby(level=0)
   
    #...calulate the median
    medians = gby.median()

    #...and the the first and third quartile
    low_q = gby.quantile(0.25)
    high_q = gby.quantile(0.75)

    #The number of trials is approximated by the number of rewards at the left port
    N_trials = len(segmented[segmented.state_name == "leftReward"].segment_no.unique())
    
    #Plot these values
    plt.figure(figsize=(7, 3))
    plt.plot(medians)
    plt.fill_between(medians.index, low_q, high_q, alpha=.2)
    plt.title(f"{sess} (N={N_trials} trials)")
    labels = ["Turn at initiation", "Run in main arm", "Turn to left", "Run in left arm", "At reward"]
    for i in range(1, 5):
        plt.axvline(i*n_bins_per_seg, color='k', linestyle='--')
    plt.xticks((np.arange(5)+0.5)*n_bins_per_seg, labels, rotation=15, ha="right")
    plt.ylabel("Average speed (cm/s)")
    plt.ylim(-5, 30)
    sns.despine()
    plt.show()

In [7]:
# list_of_avg = []
# x = 1
# dir_path = input("Insert directory path for plotting here: ")
# day = 1

# # Iterate through the particle filter and beh_segmentation lists that contain the absolute paths of the files of interest.
# for (i, j) in zip(particle_filter_list, beh_segmentation_list):
#     n_bins_per_seg = 10
#     # for sess in SessionIterator("Miniscope recordings").findSessions():
#     filtered = pd.read_hdf(i) # Particle Filter
#     segmented = pd.read_hdf(j, key="/per_frame") # Behavioral Segmentation
#     state_order_series = pd.Series(state_order, name="state_order")
#     segmented_relevant = segmented.join(state_order_series, on="state_name")
#     x_pos = segmented_relevant.spatial_progress + segmented_relevant.state_order
#     speeds = filtered.speed[np.isfinite(x_pos)]
#     bin_x = np.floor(x_pos[np.isfinite(x_pos)]*n_bins_per_seg).astype(int)
#     gby = (60*speeds).groupby([bin_x, segmented_relevant.segment_no]).mean().groupby(level=0)
#     avgs = gby.mean()
#     low_q = gby.mean()-gby.sem()
#     high_q = gby.mean()+gby.sem()
#     N_trials = len(segmented[segmented.state_name == "leftReward"].segment_no.unique())
#     avgs = avgs.clip(lower=0)
#     list_of_avg += [avgs]

#     # Plotting
#     plt.figure(figsize=(7, 3))
#     plt.plot(avgs)
#     plt.fill_between(avgs.index, low_q, high_q, alpha=.2)
#     plt.text(0.9, 0.85, f"N={N_trials} trials", ha="right", transform=plt.gca().transAxes)
#     plt.title("Animal "+str(x)+" learning day 1")
#     labels = ["Turn at initiation", "Run in main arm", "Turn to left", "Run in left arm", "At reward"]
#     for i in range(1, 5):
#         plt.axvline(i*n_bins_per_seg, color='k', linestyle='--')
#     plt.xticks((np.arange(5)+0.5)*n_bins_per_seg, labels, rotation=15, ha="right")
#     plt.ylabel("Average speed (cm/s)")
#     plt.ylim(-5, 30)
#     sns.despine()
#     #print(i, j)
#     # Saving plots
#     if x == 4:
#         x == 1
#     plt.savefig(dir_path+"/"+'animal-'+str(x)+'.png', bbox_inches='tight', dpi=300 )
#     plt.show()
#     x += 1
    

# # Average of all animals, learning day 1 with standard deviation
# total_avg = pd.concat(list_of_avg, axis=1, ignore_index=True)
# total_avg = total_avg[:-1]
# total_avg = np.mean(total_avg, axis=1)


# state_order_series = pd.Series(state_order, name="state_order")
# segmented_relevant = segmented.join(state_order_series, on="state_name")
# x_pos = segmented_relevant.spatial_progress + segmented_relevant.state_order
# speeds = filtered.speed[np.isfinite(x_pos)]
# bin_x = np.floor(x_pos[np.isfinite(x_pos)]*n_bins_per_seg).astype(int)
# gby = (60*speeds).groupby([bin_x, segmented_relevant.segment_no]).mean().groupby(level=0)
# total_avg = gby.mean() 
# N_trials = len(segmented[segmented.state_name == "leftReward"].segment_no.unique())
# plt.figure(figsize=(7, 3))
# total_avg = total_avg.clip(lower=0)
# plt.plot(total_avg)
# plt.fill_between(total_avg.index, low_q, high_q, alpha=.2)


# #plt.text(0.9, 0.85, f"N={N_trials} trials", ha="right", transform=plt.gca().transAxes)
# plt.title("All Animals / Average Speed / Learning Day 1")
# labels = ["Turn at initiation", "Run in main arm", "Turn to left", "Run in left arm", "At reward"]
# for i in range(1, 5):
#     plt.axvline(i*n_bins_per_seg, color='k', linestyle='--')
# plt.xticks((np.arange(5)+0.5)*n_bins_per_seg, labels, rotation=15, ha="right")
# plt.ylabel("Average speed (cm/s)")
# plt.ylim(-5, 40)
# sns.despine()
# plt.savefig(dir_path+"/"+'all-animals-day-'+str(day)+'.png', bbox_inches='tight', dpi=300 )
# plt.show()


# Average of all animals learning day 11

In [8]:
# # Iterate through the particle filter directory and print the pathnames of files and place them in a list
# particle_filter_list_day11 = search_particle_filter(dir_path = input("Insert directory path here: "))
# print("Files in folder: \n", particle_filter_list_day11)

In [9]:
# # Iterate behavioral segmentation directory
# beh_segmentation_list_day11 = search_behavior_segmentation(dir_path = input("Insert directory path here: "))
# print("Files in folder: \n", beh_segmentation_list_day11)

In [10]:
# list_of_avg_day11 = []
# x = 1
# day = 11
# dir_path = input("Insert directory path for plotting here: ")

# # Iterate through the particle filter and beh_segmentation lists that contain the absolute paths of the files of interest.
# for (i, j) in zip(particle_filter_list_day11, beh_segmentation_list_day11):
#     n_bins_per_seg = 10
#     # for sess in SessionIterator("Miniscope recordings").findSessions():
#     filtered = pd.read_hdf(i) # Particle Filter
#     # hf = h5py.File('data.h5', 'r')
#     #segmented = h5py.File(j)
#     segmented = pd.read_hdf(j, key="/per_frame") # Behavioral Segmentation
#     state_order_series = pd.Series(state_order, name="state_order")
#     segmented_relevant = segmented.join(state_order_series, on="state_name")
#     x_pos = segmented_relevant.spatial_progress + segmented_relevant.state_order
#     speeds = filtered.speed[np.isfinite(x_pos)]
#     bin_x = np.floor(x_pos[np.isfinite(x_pos)]*n_bins_per_seg).astype(int)
#     gby = (60*speeds).groupby([bin_x, segmented_relevant.segment_no]).mean().groupby(level=0)
#     avgs_day11 = gby.mean()
#     low_q = gby.mean()-gby.sem()
#     high_q = gby.mean()+gby.sem()
#     N_trials = len(segmented[segmented.state_name == "leftReward"].segment_no.unique())
#     avgs_day11 = avgs_day11.clip(lower=0)
#     list_of_avg_day11 += [avgs_day11]

#     # Plotting
#     plt.figure(figsize=(7, 3))
#     plt.plot(avgs_day11)
#     plt.fill_between(avgs_day11.index, low_q, high_q, alpha=.2)
#     plt.text(0.9, 0.85, f"N={N_trials} trials", ha="right", transform=plt.gca().transAxes)
#     plt.title("Animal "+str(x)+" learning day 1")
#     labels = ["Turn at initiation", "Run in main arm", "Turn to left", "Run in left arm", "At reward"]
#     for i in range(1, 5):
#         plt.axvline(i*n_bins_per_seg, color='k', linestyle='--')
#     plt.xticks((np.arange(5)+0.5)*n_bins_per_seg, labels, rotation=15, ha="right")
#     plt.ylabel("Average speed (cm/s)")
#     plt.ylim(-5, 30)
#     sns.despine()
#     #print(i, j)
#     if x == 4:
#         x == 1
#     # # Saving plots
#     plt.savefig(dir_path+"/"+'animal-'+str(x)+'.png', bbox_inches='tight', dpi=300 )
#     plt.show()
#     x += 1

    
# # Average for all animals day 11

# total_avg_day1 = pd.concat(list_of_avg_day11, axis=1, ignore_index=True)
# total_avg_day1 = total_avg_day1[:-1]
# total_avg_day1 = np.mean(total_avg_day1, axis=1)

# # remove nan values
# state_order_series = pd.Series(state_order, name="state_order")
# segmented_relevant = segmented.join(state_order_series, on="state_name")
# x_pos = segmented_relevant.spatial_progress + segmented_relevant.state_order
# speeds = filtered.speed[np.isfinite(x_pos)]
# bin_x = np.floor(x_pos[np.isfinite(x_pos)]*n_bins_per_seg).astype(int)
# gby = (60*speeds).groupby([bin_x, segmented_relevant.segment_no]).mean().groupby(level=0)
# total_avg_day1 = gby.mean()
# N_trials = len(segmented[segmented.state_name == "leftReward"].segment_no.unique())
# plt.figure(figsize=(7, 3))
# total_avg = total_avg.clip(lower=0)


# # Plotting
# plt.title("All Animals / Average Speed / Learning Day 11")
# plt.plot(total_avg_day1)
# plt.fill_between(total_avg_day1.index, low_q, high_q, alpha=.2)

# for i in range(1, 5):
#     plt.axvline(i*n_bins_per_seg, color='k', linestyle='--')
# plt.xticks((np.arange(5)+0.5)*n_bins_per_seg, labels, rotation=15, ha="right")
# plt.ylabel("Average speed (cm/s)")
# plt.ylim(-5, 40)
# sns.despine()
# plt.show()


# # Saving plots
# plt.savefig(dir_path+"/"+'all-animals-day-'+str(day)+'.png', bbox_inches='tight', dpi=300 )
# plt.show()